# LangChain Agent Conversation and Memory

Inspiration taken from https://www.youtube.com/watch?v=X550Zbz_ROE

This is important to consider because amount of tokens is not limitless meaning that we need to be wise about the way we store context during an conversation with the LLM

In [5]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

Initiate model as usual

In [6]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Look at first ConversationBufferMemory for simpler tasks

Instiate memory where log will be stored. This will be track what user & AI exchange, and stack that into prompt

In [7]:
ai_memory = ConversationBufferMemory()

Start conversational chain. This like LLMChain, can also include a prompt (let's test that in the Playground notebooks)

In [8]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=ai_memory,
    verbose=True
)

In [10]:
conversation_chain.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm El Brian
AI:

> Finished chain.


"Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and tasks. How can I help you today?"

Notice how it keeps track of the conversation even as new queries are requested

In [20]:
conversation_chain.predict(input="how is your afternoon so far?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm El Brian
AI: Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and tasks. How can I help you today?
Human: how is your afternoon so far?
AI:

> Finished chain.


"As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?"

Notice below the memory object where AI/User exchange is logged

In [22]:
print(ai_memory.chat_memory.messages)

[HumanMessage(content="Hi, I'm El Brian", additional_kwargs={}, example=False), AIMessage(content="Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and tasks. How can I help you today?", additional_kwargs={}, example=False), HumanMessage(content='how is your afternoon so far?', additional_kwargs={}, example=False), AIMessage(content="As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?", additional_kwargs={}, example=False)]


This is also stored in the actual Conversation Chain object

In [23]:
print(conversation_chain.memory.buffer)

Human: Hi, I'm El Brian
AI: Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and tasks. How can I help you today?
Human: how is your afternoon so far?
AI: As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?


The above type of memory (ConversationBufferMemory) works well where limited exchanges are expected (~5). Let's look at longer memory types below

### Let's explore ConversationSummaryMemory for some more exchange intensives scenarios

In [24]:
from langchain.memory import ConversationSummaryMemory

Notice how this initiation of conversation memory does require the llm input (unlike ConversationMemory above)

<u>Why the difference?</u> Because it uses an additional LLM to summarise the log, besides the LLM use for the  (Test multi-LLM usage in Playground)

In [26]:
ai_summary_memory = ConversationSummaryMemory(llm=llm_openai)

In [28]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=ai_summary_memory,
    verbose=True
)

In [29]:
conversation_chain.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm El Brian
AI:

> Finished chain.


"Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and answer any questions you might have. How can I assist you today?"

Notice the key summary difference now

In [30]:
conversation_chain.predict(input="How are you doing this evening?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human, El Brian, introduces himself to the AI. The AI responds by introducing itself as an assistant developed to provide information and answer questions. It then asks El Brian how it can assist him.
Human: How are you doing this evening?
AI:

> Finished chain.


"As an artificial intelligence, I don't have feelings or personal experiences, but I'm here and ready to assist you. How can I help you this evening?"

They AI summarizes the prior conversation, rather than logging the entire conversation

In [33]:
print(conversation_chain.memory.buffer)

El Brian introduces himself to the AI, which identifies itself as an assistant designed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but is ready to assist. It then reiterates its readiness to help El Brian.


Notice however in the example that at first we are using more tokens. Let's add more to the log to keep seeing how summarization works

In [34]:
conversation_chain.predict(input="Can you help me with identifying the best country to visit in the summer?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
El Brian introduces himself to the AI, which identifies itself as an assistant designed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but is ready to assist. It then reiterates its readiness to help El Brian.
Human: Can you help me with identifying the best country to visit in the summer?
AI:

> Finished chain.


'Of course, I\'d be happy to help with that! However, the "best" country to visit in the summer can depend on a variety of factors, such as your personal interests, budget, and tolerance for heat. \n\nIf you\'re a fan of mild weather, you might consider visiting Iceland. The average temperature in the summer is around 10-13 degrees Celsius (50-55 degrees Fahrenheit), and the country is known for its stunning natural beauty, including waterfalls, geysers, and hot springs.\n\nIf you\'re interested in history and culture, Italy could be a great choice. Summer is a popular time to visit cities like Rome, Florence, and Venice, where you can see famous landmarks like the Colosseum, the Uffizi Gallery, and the Grand Canal.\n\nFor beach lovers, the Maldives offers beautiful white sand beaches and clear, warm waters. The country is also known for its luxury resorts, many of which are located on their own private islands.\n\nIf you\'re a foodie, you might consider visiting Japan. The country is 

In [35]:
print(conversation_chain.memory.buffer)

El Brian introduces himself to the AI, which identifies itself as an assistant designed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but is ready to assist. El Brian then asks the AI to help identify the best country to visit in the summer. The AI responds that the "best" country can depend on various factors like personal interests, budget, and heat tolerance. It suggests Iceland for mild weather and natural beauty, Italy for history and culture, the Maldives for beaches and luxury resorts, and Japan for food lovers. The AI also offers to provide a more tailored recommendation if given more information about El Brian's preferences for a summer vacation.


The above is a nice option to store memory in summary so that the 'important' context is maximized in memory (tokens). Let's look at an alternate version of the memory where we store memory (tokens) based on a pre-defined token window

### Let's explore ConversationBufferWindowMemory to test this pre-defined token-windowed approach

In [36]:
from langchain.memory import ConversationBufferWindowMemory

The conversation buffer window memory function sets the last number of interactions we want to keep. We are using 2 as an example below

In [37]:
window_memory = ConversationBufferWindowMemory(k=2)

We pass it to the conversation chain memory argument (as before)

In [38]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=window_memory,
    verbose=True
)

In [39]:
conversation_chain.predict(input="I'm looking for travel support")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I'm looking for travel support
AI:

> Finished chain.


"Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. \n\n1. Destination Research: I can provide information about various destinations, including popular tourist spots, local cuisine, culture, and more.\n\n2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.\n\n3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.\n\n4. Travel Tips: I can provide tips on packing, local customs, currency exchange, and more.\n\n5. Emergency Support: I can provide information on what to do in case of emergencies, including local emergency numbers, nearest hospitals, etc.\n\nPlease let me know which area you need help with, or if there's something specific you're looking for."

In [40]:
conversation_chain.predict(input="I would like suggestions for the best places to travel in the US with a low budget")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including popular tourist spots, local cuisine, culture, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local customs, currency exchange, and more.

5. Emergency Support: I can provide i

"Absolutely, I'd be happy to provide some suggestions. Traveling on a budget in the US is definitely possible, and there are many beautiful and affordable destinations to choose from. Here are a few:\n\n1. **Yellowstone National Park, Wyoming**: This is one of the most famous national parks in the US. You can camp, hike, and enjoy the natural beauty. The entrance fee is relatively low, and camping can be a cost-effective accommodation option.\n\n2. **Austin, Texas**: Known for its vibrant music scene and delicious food, Austin can be a great destination for budget travelers. There are many free or low-cost attractions like Zilker Park and the State Capitol.\n\n3. **Savannah, Georgia**: This charming southern city offers beautiful parks, historic architecture, and delicious food. Many of the city's attractions are free or low-cost.\n\n4. **Portland, Oregon**: Known for its food trucks, microbreweries, and beautiful parks, Portland can be a great destination for budget travelers. The cit

Notice that as before (non-summary chain) we have our two past outputs

In [41]:
print(conversation_chain.memory.buffer)

Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including popular tourist spots, local cuisine, culture, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local customs, currency exchange, and more.

5. Emergency Support: I can provide information on what to do in case of emergencies, including local emergency numbers, nearest hospitals, etc.

Please let me know which area you need help with, or if there's something specific you're looking for.
Human: I would like suggestions for the best places to travel in the US with a low budget
AI: Absolutely, I'd b

In [42]:
conversation_chain.predict(input="Where should I stay in Austin Texas?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including popular tourist spots, local cuisine, culture, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local customs, currency exchange, and more.

5. Emergency Support: I can provide i

"Austin, Texas, has a variety of neighborhoods that cater to different interests and budgets. Here are a few suggestions:\n\n1. **Downtown Austin**: This is the heart of the city, where you'll find a lot of the action. It's home to many of Austin's famous music venues, restaurants, and bars. There are a variety of hotels here, ranging from luxury to budget.\n\n2. **East Austin**: This is a trendy, up-and-coming area with a lot of character. It's known for its food scene, with many food trucks and unique restaurants. There are a variety of accommodation options here, including boutique hotels and budget-friendly hostels.\n\n3. **South Congress (SoCo)**: This area is known for its shopping, dining, and entertainment. It's a great place to stay if you want to experience Austin's quirky side. Accommodations here range from stylish boutique hotels to more affordable motels.\n\n4. **The University of Texas area**: If you're looking for a more budget-friendly area, the vicinity around the Uni

Notice now that it has lost the first interaction. <u>This is what window memory buffer does</u>, it allows the user to manually set how many windows of memory to keep, therefore reducing the amount of tokens required at the end

In [43]:
print(conversation_chain.memory.buffer)

Human: I would like suggestions for the best places to travel in the US with a low budget
AI: Absolutely, I'd be happy to provide some suggestions. Traveling on a budget in the US is definitely possible, and there are many beautiful and affordable destinations to choose from. Here are a few:

1. **Yellowstone National Park, Wyoming**: This is one of the most famous national parks in the US. You can camp, hike, and enjoy the natural beauty. The entrance fee is relatively low, and camping can be a cost-effective accommodation option.

2. **Austin, Texas**: Known for its vibrant music scene and delicious food, Austin can be a great destination for budget travelers. There are many free or low-cost attractions like Zilker Park and the State Capitol.

3. **Savannah, Georgia**: This charming southern city offers beautiful parks, historic architecture, and delicious food. Many of the city's attractions are free or low-cost.

4. **Portland, Oregon**: Known for its food trucks, microbreweries, a

Let's look at now an alternative version, which is a combination of memory buffer of sorts, which mixes a max token limit over re-created summaries

### Let's explore ConversationSummaryBufferMemory which applies a maximum token threshold over summaries

In [44]:
from langchain.memory import ConversationSummaryBufferMemory

Let's load the memory class, and as before with ConversationSummaryMemory(), notice that it needs an llm to also summarize context. Rather than a window also notice that we are using max tokens to threshold memory usage

In [46]:
memory = ConversationSummaryBufferMemory(llm=llm_openai,
                                          max_token_limit=50)

In [47]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=memory,
    verbose=True
)

In [48]:
conversation_chain.predict(input="Hi, this is El Brain")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, this is El Brain
AI:

> Finished chain.


"Hello El Brain! It's a pleasure to meet you. I'm an AI developed to assist with a variety of tasks and answer any questions you might have to the best of my ability. How can I assist you today?"

In [49]:
conversation_chain.predict(input="I need help with travel arrengement within the US")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as El Brain.
AI: Hello El Brain! It's a pleasure to meet you. I'm an AI developed to assist with a variety of tasks and answer any questions you might have to the best of my ability. How can I assist you today?
Human: I need help with travel arrengement within the US
AI:

> Finished chain.


"Of course, El Brain! I'd be happy to assist you with your travel arrangements. Could you please provide me with more details? For instance, your departure and destination cities, preferred dates of travel, whether you need a round trip or one-way ticket, and any other preferences you might have such as direct flights, specific airlines, or budget constraints."

In [54]:
conversation_chain.predict(input="I want to travel over the summer within low budget")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: El Brain introduces themselves to the AI, which responds by offering its assistance with a variety of tasks. El Brain asks for help with travel arrangements within the US, and the AI asks for more details such as departure and destination cities, dates of travel, type of ticket, and any other preferences.
Human: I want to travel over the summer within low budget
AI:

> Finished chain.


"That sounds like a great plan! To help you better, I'll need a bit more information. Could you please provide:\n\n1. Your departure city and destination city.\n2. The specific dates or range of dates you're considering for your travel.\n3. Any flexibility you might have with your travel dates.\n4. Your preference for direct flights or if you're open to flights with stops (which can sometimes be cheaper).\n5. Any airline preference you might have.\n6. If you have any hotel preferences or if you're open to suggestions.\n7. Any other specific requirements or preferences you might have, such as rental car needs, dietary restrictions for airline meals, etc.\n\nWith these details, I can provide a more accurate and tailored travel plan for you."

In [64]:
conversation_chain.predict(input="I'm looking into cities in the mid west")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: El Brain introduces themselves to the AI, which responds by offering its assistance with a variety of tasks. El Brain asks for help with travel arrangements within the US, and the AI asks for more details such as departure and destination cities, dates of travel, type of ticket, and any other preferences. El Brain specifies they want to travel over the summer on a low budget. The AI then asks for more specific information including departure and destination cities, specific dates or range of dates, flexibility with travel dates, preference for direct flights or flights with stops, airline preference, hotel preferences, and any other specific requirements or pre

"That's great! The Midwest has a lot to offer. Some popular cities to visit are Chicago, Illinois; Minneapolis, Minnesota; Indianapolis, Indiana; and Kansas City, Missouri. Each of these cities has its own unique attractions and charm. Do you have any specific cities in mind or are you open to suggestions? Also, do you have any specific interests or activities you would like to do during your visit? This will help me provide more tailored recommendations."

Notice two things. The return object contain within the conversation chain is "moving_summary_buffer" rather than "buffer" AND the summary window is constrained to the 50 token limit

In [70]:
print(conversation_chain.memory.moving_summary_buffer)

El Brain introduces themselves to the AI, which offers its assistance with a variety of tasks, including travel arrangements within the US. El Brain wants to travel over the summer on a low budget and is looking into cities in the Midwest. The AI suggests popular cities like Chicago, Minneapolis, Indianapolis, and Kansas City, and asks for specific interests or activities El Brain would like to do during the visit for more tailored recommendations. The AI also asks for more specific information including departure and destination cities, specific dates or range of dates, flexibility with travel dates, preference for direct flights or flights with stops, airline preference, hotel preferences, and any other specific requirements or preferences such as rental car needs or dietary restrictions for airline meals.


This is a great way to summarize memory with token limit. Let's look at alternative more algorithmically way to handle memory

### Let's take a look at Conversation Knowledge Graph Memory (ConversationKGMemory)

In [71]:
from langchain.memory import ConversationKGMemory